

QUESTIONS:
What is our 30-day all cause readmission rate across hospitals our members are admitted to?
What percent of patients have a PCP visit within a week of discharge?
Do either vary by individual or group PCP's?

BONUS:
Which individual or PCP groups should we make an extra effort with to build a relationship? Which have the highest volume?
What percent of our patients do not have a PCP assigned or haven't seen their PCP recently?
Are there certain patients we should focus on (e.g. frequent fliers)?
From this, what could a version 1 of an actionable dashboard or intervention list look like to support the Transition Care Managers?

NOTES:

discharge date + 30 days: readmission true or false
count trues

In [2]:
import pandas as pd
import datetime as dt
import requests

In [46]:
members = pd.read_csv('../Data Files/NSS_GreatCare_Members.csv', parse_dates=[61,62], dtype={'Empirically Attributed PCP Individual Provider NPI':'object', 'Empirically Attributed PCP Group Provider NPI':'object'})
admissions = pd.read_csv('../Data Files/NSS_Hospital_Inpatient_Admissions.csv', parse_dates=[8,9])
pcp_visits = pd.read_csv('../Data Files/NSS_PrimaryCare_Office_Visits.csv', parse_dates=[12], dtype={'Service Provider NPI':'object', 'Provider Group NPI':'object'})

In [4]:
admissions_dedup = admissions.sort_values(by=['Patient ID','Encounter Start Date', 'Encounter End Date']).drop_duplicates(subset=['Patient ID', 'Encounter Start Date'], keep = 'last')
admissions_dedup = admissions_dedup.sort_values(by=['Patient ID','Encounter End Date', 'Encounter Start Date']).drop_duplicates(subset=['Patient ID', 'Encounter End Date'], keep = 'first')
admissions.shape, admissions_dedup.shape

((6213, 23), (6149, 23))

In [5]:
def categorize_events(patient_admissions):
    patient_index_events = 0
    patient_readmission_events = 0
    last_index_event_discharge_date = None
    last_index_event_encounter_id = None
    readmission = False
    for ind, row in patient_admissions.iterrows():
        ##set first event as an intiial event or a non-qualifying readmission as initial event
        if patient_index_events == 0 or row['Encounter Start Date'] > last_index_event_discharge_date + dt.timedelta(days=30):
            patient_index_events += 1
            last_index_event_discharge_date = row['Encounter End Date']
            last_index_event_encounter_id = row['Encounter ID']
            readmission = False
            admissions.loc[admissions['Encounter ID'] == row['Encounter ID'], 'type'] = 'index without readmission'
        ##check if qualifying readmission and change readmit status of most recently added initial event to true
        elif readmission == False:
            patient_readmission_events += 1
            readmission = True
            admissions.loc[admissions['Encounter ID'] == last_index_event_encounter_id, 'type'] = 'index w readmission'
            admissions.loc[admissions['Encounter ID'] == row['Encounter ID'], 'type'] = 'initial readmission'
        ##check for duplicate readmission
        else:
            admissions.loc[admissions['Encounter ID'] == row['Encounter ID'], 'type'] = 'extra readmission'
            
    return patient_index_events, patient_readmission_events
    

In [6]:
total_index_events = 0
total_index_events_leading_to_readmission = 0

for patient in admissions_dedup['Patient ID'].unique():
    patient_admissions = admissions_dedup[admissions['Patient ID']==patient]
    times_admitted = patient_admissions.shape[0]
    if times_admitted > 1:
        patient_admissions.sort_values(by = 'Encounter End Date', inplace=True, ignore_index=True)
        ind, re = categorize_events(patient_admissions)
        total_index_events += ind
        total_index_events_leading_to_readmission += re
    else:
        encounter_id = patient_admissions['Encounter ID'].values[0]
        admissions.loc[admissions['Encounter ID'] == encounter_id, 'type'] = 'index without readmission'
        total_index_events += 1
print("Total Index Events:",total_index_events) 
print("Index Events Leading to Readmission:", total_index_events_leading_to_readmission)
print(str.format("Readmission Rate: {}%",round(total_index_events_leading_to_readmission/total_index_events*100,2)))

<ipython-input-6-45bfff96a76a>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  patient_admissions = admissions_dedup[admissions['Patient ID']==patient]
<ipython-input-6-45bfff96a76a>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patient_admissions.sort_values(by = 'Encounter End Date', inplace=True, ignore_index=True)


Total Index Events: 5536
Index Events Leading to Readmission: 527
Readmission Rate: 9.52%


In [7]:
admissions['type'].value_counts()

index without readmission    5009
index w readmission           527
initial readmission           527
extra readmission              86
Name: type, dtype: int64

In [8]:
admissions[['type']] = admissions[['type']].fillna(value='duplicate encounter')

In [9]:
admissions['type'].value_counts()

index without readmission    5009
index w readmission           527
initial readmission           527
extra readmission              86
duplicate encounter            64
Name: type, dtype: int64

In [10]:
total_hospital_visits = admissions_dedup.shape[0]
total_pcp_visits_in_7_days = 0

for ind, row in admissions_dedup.iterrows():
    patient = row['Patient ID']
    discharge_date = row['Encounter End Date']
    
    for i,r in pcp_visits[pcp_visits['Patient ID']==patient].iterrows():
        if r['Encounter Date'] <= discharge_date + dt.timedelta(days=7) and r['Encounter Date'] >= discharge_date:
            total_pcp_visits_in_7_days += 1
            admissions.loc[admissions['Encounter ID']==row['Encounter ID'], '7 day followup'] = True
            break
            
print('Total Hospital Visits: ', total_hospital_visits)
print('Total PCP Visits in 7 Days: ', total_pcp_visits_in_7_days)
print(str.format("PCP Visit Rate: {}%",round(total_pcp_visits_in_7_days/total_hospital_visits*100,2)))


Total Hospital Visits:  6149
Total PCP Visits in 7 Days:  1860
PCP Visit Rate: 30.25%


In [11]:
for ind, row in admissions.iterrows():
    patient = row['Patient ID']
    discharge = row['Encounter End Date']
    encounter = row['Encounter ID']
    mask = (pcp_visits['Patient ID'] == patient) & (pcp_visits['Encounter Date']>= discharge)
    patient_pcp = pcp_visits[mask].sort_values('Encounter Date')
    if len(patient_pcp['Encounter Date']) == 0:
        continue
    else:
        first_pcp_visit = patient_pcp.head(1)
        indiv_npi = first_pcp_visit['Service Provider NPI'].values[0]
        group_npi = first_pcp_visit['Provider Group NPI'].values[0]
        if indiv_npi == group_npi:
            admissions.loc[admissions['Encounter ID']==encounter, 'individual_npi'] = indiv_npi
        else:
            admissions.loc[admissions['Encounter ID']==encounter, 'individual_npi'] = indiv_npi
            admissions.loc[admissions['Encounter ID']==encounter, 'group_npi'] = group_npi

In [12]:
admissions[['7 day followup']] = admissions[['7 day followup']].fillna(value=False)
admissions.head()

,Encounter ID,Patient ID,Encounter Primary Payor ID,Encounter Primary Plan ID,Facility Provider ID,Attending Provider ID,Attending Provider NPI,Facility Provider NPI,Encounter Start Date,Encounter End Date,...,Principal Discharge Diagnosis ICD-9-CM Description,Principal Discharge Diagnosis ICD-10-CM Code,Principal Discharge Diagnosis ICD-10-CM Description,Primary Plan Allowed Amount,Primary Plan Paid Amount,Patient Paid Amount,type,7 day followup,individual_npi,group_npi
0,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,DS1|1,DS1|5047W,DS1|NPI|1306890389,DS1|NPI|1417170523,1.417171e+09,1306890389,2013-01-12,2013-01-14,...,NaN,O70.0,First degree perineal laceration during delivery,0,6620.72,250.0,index without readmission,False,1619980406,1043427248
1,HIA|DS1|10043|DS1|NPI|1902803315|1,DS1|10043,DS1|1,DS1|5047Y,DS1|NPI|1902803315,DS1|NPI|1669599197,1.669599e+09,1902803315,2014-01-05,2014-01-12,...,Compression of brain,NaN,NaN,0,46465.40,250.0,index without readmission,True,1619199791,1043427248
2,HIA|DS1|10073|DS1|NPI|1902803315|1,DS1|10073,DS1|1,DS1|5047X,DS1|NPI|1902803315,DS1|NPI|1407880511,1.407881e+09,1902803315,2012-06-13,2012-06-21,...,NaN,K91.71,Accidental puncture and laceration of a digest...,0,43879.13,250.0,index without readmission,True,1659394021,1043427248
3,HIA|DS1|10101|DS1|NPI|1427055839|1,DS1|10101,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1407008907,1.407009e+09,1427055839,2012-12-06,2012-12-09,...,NaN,O70.1,Second degree perineal laceration during delivery,0,14800.00,250.0,index without readmission,True,1952538340,1043427248
4,HIA|DS1|10118|DS1|NPI|1992818256|1,DS1|10118,DS1|1,DS1|5047W,DS1|NPI|1992818256,DS1|NPI|1518045608,1.518046e+09,1992818256,2015-03-11,2015-03-14,...,NaN,F32.9,"Major depressive disorder, single episode, uns...",0,0.00,0.0,index without readmission,False,1316361942,1043427248


In [27]:
print(admissions['type'].value_counts())
admissions['individual_npi'].isna().sum(), admissions['group_npi'].isna().sum()

index without readmission    5009
index w readmission           527
initial readmission           527
extra readmission              86
duplicate encounter            64
Name: type, dtype: int64


(1288, 1380)

In [36]:
individual_mask = ~admissions['individual_npi'].isna()
index_events = admissions[individual_mask]['type'].value_counts()[0] + admissions[individual_mask]['type'].value_counts()[1]
readmissions = admissions[individual_mask]['type'].value_counts()[1]

non_dup = ~(admissions['type']=='duplicate encounter') & individual_mask
admission_events = admissions[non_dup].shape[0]
followup_7_days = admissions[(non_dup) & (admissions['7 day followup']==True)].shape[0]

print(f'Readmission rate for patients who see an individual PCP after discharge: {round(readmissions/index_events*100,2)}%')
print(f'7 day followup rate for patients who see an individual PCP after discharge: {round(followup_7_days/admission_events*100,2)}%')

Readmission rate for patients who see an individual PCP after discharge: 9.67%
7 day followup rate for patients who see an individual PCP after discharge: 38.01%


In [37]:
group_mask = ~admissions['group_npi'].isna()
index_events_g = admissions[group_mask]['type'].value_counts()[0] + admissions[group_mask]['type'].value_counts()[1]
readmissions_g = admissions[group_mask]['type'].value_counts()[1]

non_dup_g = ~(admissions['type']=='duplicate encounter') & group_mask
admission_events_g = admissions[non_dup_g].shape[0]
followup_7_days_g = admissions[(non_dup_g) & (admissions['7 day followup']==True)].shape[0]

print(f'Readmission rate for patients who see a group PCP after discharge: {round(readmissions_g/index_events_g*100,2)}%')
print(f'7 day followup rate for patients who see a group PCP after discharge: {round(followup_7_days_g/admission_events_g*100,2)}%')

Readmission rate for patients who see a group PCP after discharge: 9.63%
7 day followup rate for patients who see a group PCP after discharge: 38.21%


In [25]:
admissions['group_npi'].value_counts()

1043427248    2209
1184722779    1164
1164512851     481
1407058787     247
1316025091     173
              ... 
1073580320       1
1275975104       1
1093106569       1
1043628316       1
1881810042       1
Name: group_npi, Length: 116, dtype: int64

In [41]:
admissions['individual_npi'].value_counts()

1104884337    66
1619980406    52
1467643700    50
1972704856    50
1437180965    49
              ..
1942315387     1
1528196185     1
1811192651     1
1033404645     1
1235154618     1
Name: individual_npi, Length: 979, dtype: int64

In [42]:
admissions['Patient ID'].value_counts()

DS1|3812     17
DS1|26751    15
DS1|53241    13
DS1|10451    12
DS1|12422    11
             ..
DS1|22718     1
DS1|58843     1
DS1|41030     1
DS1|39166     1
DS1|6550      1
Name: Patient ID, Length: 4701, dtype: int64

In [47]:
members['Empirically Attributed PCP Individual Provider NPI'].isna().sum()

29680

In [48]:
members['Empirically Attributed PCP Group Provider NPI'].isna().sum()

29631

In [55]:
mask = members['Empirically Attributed PCP Individual Provider NPI'].isna() & members['Empirically Attributed PCP Group Provider NPI'].isna()
print(f'Members who do not have an assigned PCP (group or individual): {round(members[mask].shape[0]/members.shape[0]*100,2)}%')

Members who do not have an assigned PCP (group or individual): 47.96%


In [ ]:
for member in members['Patient ID'].unique():
    member_pcp_visits = pcp_visits[pcp_visits['Patient ID']==member]
    most_recent_visit = member_pcp_visits['Encounter Date'].max()
    members.loc[members['Patient ID']==member, 'most recent visit'] = most_recent_visit

In [58]:
members

,Patient ID,Primary Plan ID,Primary Payor ID,Is Patient Alive,Patient Age Integer,Patient Date of Birth,Patient ZIP Code 5-Digit,Count Primary Care Visits in Last 36 Months,Count Visits with Attributee Individual Primary Care Provider i,Count Visits with Attributee Primary Care Provider Group in Las,...,Is CCW Rheumatoid Arthritis or Osteoarthritis,Is CCW Stroke or Transient Ischemic Attack,Is CCW ASCVD,Is CCW Anxiety Disorders,Is CCW Depressive Disorders,Is CCW Schizophrenia or Other Psychotic Disorders,Is CCW Bipolar Disorder,Period Start Date,Period End Date,most recent visit
0,DS1|1,DS1|5047W,DS1|1,1,54,1958-11-08,92111.0,0,NaN,NaN,...,0,0,0,0,0,0,0,2013-01-01 00:00:00+00:00,2013-04-11 00:00:00+00:00,2013-11-21
1,DS1|10,DS1|5047W,DS1|1,1,54,1958-12-10,90024.0,6,1.0,5.0,...,0,0,0,1,0,0,0,2012-12-10 00:00:00+00:00,2013-01-01 00:00:00+00:00,2013-06-13
2,DS1|100,DS1|5047W,DS1|1,1,2,2012-08-04,90008.0,1,1.0,1.0,...,0,0,0,0,0,0,0,2015-04-18 00:00:00+00:00,2015-05-18 00:00:00+00:00,2015-12-27
3,DS1|1000,DS1|5047W,DS1|1,1,6,2003-12-25,94131.0,0,NaN,NaN,...,0,0,0,0,0,0,0,2010-01-01 00:00:00+00:00,2010-04-09 00:00:00+00:00,2015-05-03
4,DS1|10000,DS1|5047W,DS1|1,1,7,2005-07-15,90301.0,0,NaN,NaN,...,0,0,0,0,0,0,0,2012-08-01 00:00:00+00:00,2012-10-02 00:00:00+00:00,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61514,DS1|9995,DS1|5047W,DS1|1,1,47,1963-07-26,92104.0,0,NaN,NaN,...,0,0,0,0,0,0,0,2011-01-01 00:00:00+00:00,2011-01-15 00:00:00+00:00,NaT
61515,DS1|9996,DS1|5047X,DS1|1,1,26,1983-04-17,94122.0,0,NaN,NaN,...,0,0,0,0,0,0,0,2010-03-28 00:00:00+00:00,2010-04-17 00:00:00+00:00,NaT
61516,DS1|9997,DS1|5047W,DS1|1,1,4,2008-12-11,92840.0,1,1.0,1.0,...,0,0,0,0,0,0,0,2013-01-01 00:00:00+00:00,2013-02-01 00:00:00+00:00,NaT
61517,DS1|9998,DS1|5047X,DS1|1,1,63,1954-05-23,94939.0,1,1.0,1.0,...,0,0,0,0,0,0,0,2017-11-05 00:00:00+00:00,2018-05-23 00:00:00+00:00,NaT
